# Single instance

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations, FFTW
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
T_final = 32000.

# number of points to get Lyapunov exponent for
npts = 256
# npts = 2048

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
# N = 9
#small 
# seed = 12778 # => 0.6085
# seed = 73014 # => 0.7356
# seed = 65343 # => 0.7854
# seed = 57220 # => 0.8052
# seed = 47297 # => 0.8217
# seed = 78653 # => 0.8303
# seed = 46944 # => 0.8403
# seed = 70592 # => 0.8467
# seed = 62565 # => 0.8595
# seed = 87929 # => 0.8867
# seed = 39879 # => 1.0371
# seed = 63685 # => 1.2428
seed = 6
# seed = 42
# seed = 4147
seed = 51411
# seed = 6091
# seed = 6150
seed = 23320 # late first order - GREAT ILLUSTRATION
# seed = 24423 # first order
seed = 38906 # first order - GREAT ILLUSTRATION
# seed = 50947 # first order
# seed = 41908 # degenerate gs with large gap above

# large
# seed = 2100

# N = 11
# seed = 40
# seed = 168
# seed = 684 # EXCELLENT ILLUSTRATION!

# N = 15
# seed = "36"
# seed = "325"
# seed = "19015"
# seed = "19134"
# seed = "20484"

# # N = 17 
# seed = 19074
# seed = 19141 # works well
# # seed = 194
# # seed = 2026
# # seed = 177

# N = 19
# small
# seed = 538 # => 0.4583
# seed = 628 # => 0.4771
# seed = 147 # => 0.5004
# seed = 63 # => 0.5323
# seed = 914 # => 0.5950
# seed = 863 # => 0.7336
# seed = 699 # => 0.7725
# seed = 1238 # => 0.7753
# seed = 18 # => 0.8269
# seed = 319 # => 0.8990 - GREAT ILLUSTRATION!
# seed = 674 # => 0.9061
# seed = 554 # => 1.0416

# seed = 1111 # degenerate gs with large gap above

# seed = 192

# large
# seed = 66

J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
mf_problem = Problem(0, J_mat);

## Exact gap

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap) 
gaploc = exact_times[gap_idx] 
gaploc |> println

In [ ]:
small_idxs = findall(x -> x < 0.05, gap ./ (2pi)) 
small_idxs |> println

gap_interval = exact_times[small_idxs] # exact_times[small_idxs[1:findfirst(x -> x == gap_idx, small_idxs) + 1]]
gap_interval |> println

In [ ]:
# bogo_spec = bogoliubov_spectrum(mf_problem, LyapunovParameters(32000, 32, tol, tol))
# bogo_spec = reduce(hcat, bogo_spec)
# bogo_spec = sort(bogo_spec .|> real, dims=1);

bogo_spec = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", 32000, log10(1e-8), 32));

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(bogo_spec)[1]
    # plot(exact_times[2:end], bogo_spec[i, :], "-k", ms=2)
    # plot(exact_times, (λ[i, :] .- λ[1, :]), "--r", ms=2)
    plot(exact_times[2:end], bogo_spec[i, :] ./ 2pi, "--r", ms=2)
    plot(exact_times, (λ[i, :] .- λ[1, :]) ./ 2pi, "-k", ms=2)    
end
axvline(gap_interval[1], c="C0")
axvline(gap_interval[end], c="C0")
xlim(0., 1.)
ylim(-0., 0.5)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

### Eigenstates

In [ ]:
# nev = 20
# eigeninfo = map(s -> (eigs(-SpinFluctuations.hamiltonian(1 - s, s, mf_problem.local_fields, mf_problem.couplings), nev=nev, which=:LM, maxiter=10000)), exact_times)
# λs = [vals[1] for vals in eigeninfo]
# # λ_new = sort(reduce(hcat, λ_new), dims=1);

# all_eigvecs = zeros(length(exact_times), 2^(N-1), 3)
# for k in 1:length(exact_times)
#     sorting_perm = sortperm(λs[k])
#     all_eigvecs[k, :, :] .= eigeninfo[k][2][:, sorting_perm[1:3]]
# end

In [ ]:
all_eigvecs = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_lowest_eigvecs");

In [ ]:
H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings);

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]

overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)];

In [ ]:
figure(figsize=(7, 5))
ax = subplot(221)
# plot(exact_times, abs.(overlap_01_x), label="\$X\$")
# plot(exact_times, abs.(overlap_01_z), label="\$Z\$")
plot(exact_times, abs.(overlap_01_z .- overlap_01_x), label="\$|\\langle 1|H_Z-H_X|0\\rangle|\$")
axvline(gap_interval[1], c="k")
axvline(gaploc, c="C3")
axvline(gap_interval[end], c="k")
xlim(0., 1.)
# ylim(-0., 0.5)
ax.set_xticklabels([])
ylabel("Numerator Adiab. Thm.")
legend(frameon=false)

ax = subplot(222)
plot(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])))
# semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])))
axvline(gap_interval[1], c="k")
axvline(gaploc, c="C3")
axvline(gap_interval[end], c="k")
xlim(0., 1.)
ylim(0, )
ax.set_xticklabels([])
ylabel("Adiab. Thm.")


ax = subplot(223)

# plot(exact_times, abs.(overlap_02_x), "-", label="\$X\$")
# plot(exact_times, abs.(overlap_02_z), "-", label="\$Z\$")
plot(exact_times, abs.(overlap_02_z .- overlap_02_x), label="\$|\\langle 1|H_Z-H_X|0\\rangle|\$")

axvline(gap_interval[1], c="k")
axvline(gaploc, c="C3")
axvline(gap_interval[end], c="k")
xlim(0., 1.)
# ylim(-0., 0.5)
xlabel("\$s\$")
ylabel("Numerator Adiab. Thm.")
legend(frameon=false)

ax = subplot(224)
plot(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])))
axvline(gap_interval[1], c="k")
axvline(gaploc, c="C3")
axvline(gap_interval[end], c="k")
xlim(0., 1.)
# ylim(1e-6, 1e0)
xlabel("\$s\$")
ylabel("Adiab. Thm.")

tight_layout()

## Mean-field trajectories

In [ ]:
# T_final = 1000. 
# T_final = 4000. 
# T_final = 8000. 
# T_final = 16000.
T_final = 32768.
# T_final = 64000.
tol = 1e-6;

In [ ]:
# sol = evolve_mean_field(mf_problem.local_fields, mf_problem.couplings, T_final, t -> t/T_final, rtol=1e2*tol, atol=tol)

sol_t = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
sol_u = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)));

In [ ]:
size(sol_u)

In [ ]:
component_dict = Dict("x" => 1, "y" => 2, "z" => 3)
component = "z"
# test_data = reduce(hcat, [sol.u[k][component_dict[component], :] for k in 1:size(sol.u)[1]]);
test_data = reduce(hcat, [sol_u[k, component_dict[component], :] for k in 1:size(sol_u)[1]]);

In [ ]:
areas = Dict()
# dts = [(x[2] - x[1]) / T_final for x in zip(sol.t[1:end-1], sol.t[2:end])]
dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
for spin_idx in 1:N-1
    areas[spin_idx] = sum(dts .* test_data[spin_idx, 2:end]) |> abs
end

top_idxs = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
top_idx = top_idxs[1]

In [ ]:
figure(figsize=(4, 3))

subplot(111)
# for spin_nr in 1:N-1
#     if round(abs.(test_data[spin_nr, :][end]), digits=3) == 1.0
#         plot(sol.t ./ T_final, test_data[spin_nr, :], "-")
#     else
#         plot(sol.t ./ T_final, test_data[spin_nr, :], "-", label=@sprintf("\$i=%i\$", spin_nr))    
#     end
# end

for (spin_nr, _) in sort(areas |> collect, by=x->x[2])[1:4]
    plot(sol_t ./ T_final, test_data[spin_nr, :], "-", label=@sprintf("\$i=%i\$", spin_nr))    
end

xlim(0, 1)
ylim(-1, 1)
xlabel("\$s\$")
ylabel("\$n_i^z(s)\$")
legend(frameon=false, ncol=2)

tight_layout()
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
# sigma_star = sign.(sol.u[end][3, :])
sigma_star = sign.(sol_u[end, 3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
for spin_idx in 1:N-1
    # sigma_star_flip = sign.(sol.u[end][3, :])
    sigma_star_flip = sign.(sol_u[end, 3, :])
    sigma_star_flip[spin_idx] = (-1) * sigma_star_flip[spin_idx]
    E_flip = sum([-h[l] * sigma_star_flip[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star_flip[i] * sigma_star_flip[j] for i in 1:N-1 for j in (i+1):N-1])
    println(spin_idx, ": ", E_flip)
end

In [ ]:
λ[1, :][end]

## Statistical Green function

In [ ]:
tol = 1e-8
T_final = 32000.
npts = 2048

T_final = 32768.
npts = 4096
coarse_times = range(0, 1, npts + 1);

In [ ]:
# lyapunov_parameters = LyapunovParameters(T_final, npts, 1e-4, 1e-6)
# mf_sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters);
# flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2
# all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
# F = evolve_statistical_function(mf_problem, LyapunovParameters(T_final, npts, 1e-6, 1e-6))

In [ ]:
mapped_F = F[1].(T_final .* coarse_times)
new_all_flucs = [(real.(1.0im .* diag(mapped_F[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2 for k in 1:npts+1]
new_all_flucs = reduce(hcat, new_all_flucs);

In [ ]:
all_flucs = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(1e-8), npts));

In [ ]:
function moving_average(vs, n)
    res = similar(vs, length(vs) - (n-1))
    @inbounds for i in 1:length(res)
        res[i] = sum(@view vs[i:(i + n-1)]) / n
    end
    return res
end

function smoothen(data, coarse_times; navg=64)
    ninterp = size(coarse_times)[1] - 1
    avg_data = linear_interpolation(range(0, 1, ninterp + 1)[1:end - navg + 1], moving_average(data, navg), extrapolation_bc=Line())
    map(x -> avg_data(x - navg / 2ninterp), coarse_times)
end

In [ ]:
figure(figsize=(8, 2))
subplot(131)
plot(coarse_times, mean(all_flucs, dims=1)[1, :])
# plot(coarse_times, smoothen(mean(all_flucs, dims=1)[1, :], coarse_times))

# plot(coarse_times, mean(new_all_flucs, dims=1)[1, :])

# for idx in top_idxs[1:1]
#     plot(coarse_times, new_all_flucs[idx, :] .- mean(new_all_flucs, dims=1)[1, :])
#     plot(coarse_times, all_flucs[idx, :] .- mean(all_flucs, dims=1)[1, :])
# end
axvline(gaploc, c="r")
axvline(gap_interval[1], c="k")
axvline(gap_interval[end], c="k")

subplot(132)
plot(coarse_times, sum([(all_flucs[top_idxs[k], :] .- mean(all_flucs, dims=1)[1, :]) ./ maximum(all_flucs[top_idxs[k], :]) for k in 1:3]))
axvline(gaploc, c="r")
axvline(gap_interval[1], c="k")
axvline(gap_interval[end], c="k")
ylim(0, )

subplot(133)
plot(coarse_times, all_flucs[top_idxs[1], :] .- mean(all_flucs, dims=1)[1, :])
plot(coarse_times, all_flucs[top_idxs[2], :] .- mean(all_flucs, dims=1)[1, :])
axvline(gaploc, c="r")
axvline(gap_interval[1], c="k")
axvline(gap_interval[end], c="k")
ylim(0, )
tight_layout()

In [ ]:
figure(figsize=(20, 4))
ylims = (-0.0, ) # 0.25
for i in 1:(N-1)÷2
    ax = subplot(2, (N-1)÷2, i)
    # plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    c = "C0"
    if i in top_idxs[1:3]
        c = "C3"
    end
    plot(coarse_times, all_flucs[i, :] .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)), c=c)
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times) .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)), c="k")
    axvline(gaploc, c="r")
    axvline(gap_interval[1], c="k")
    axvline(gap_interval[end], c="k")
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(ylims...)
    if i > 1
        ax.set_yticklabels([])
    end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in (N-1)÷2+1:N-1
    ax = subplot(2, (N-1)÷2, i)
    # plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    c = "C0"
    if i in top_idxs[1:3]
        c = "C3"
    end
    plot(coarse_times, all_flucs[i, :] .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)), c=c)
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times) .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)), c="k")
    axvline(gaploc, c="r")    
    axvline(gap_interval[1], c="k")
    axvline(gap_interval[end], c="k")
    xlim(0., 1.)
    ylim(ylims...)
    if i > (N-1)÷2+1
        ax.set_yticklabels([])
    end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, (N-1)÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
figure(figsize=(20, 4))
ylims = () #(-0.0, 0.1) # 0.25
for i in 1:(N-1)÷2
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    # plot(coarse_times, all_flucs[i, :] .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)))
    axvline(gaploc, c="r")
    axvline(gap_interval[1], c="k")
    axvline(gap_interval[end], c="k")
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(ylims...)
    # if i > 1
    #     ax.set_yticklabels([])
    # end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in (N-1)÷2+1:N-1
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    # plot(coarse_times, all_flucs[i, :] .- mean(all_flucs, dims=1)[1, :], label=@sprintf("\$i=%s\$", string(i)))
    axvline(gaploc, c="r")    
    axvline(gap_interval[1], c="k")
    axvline(gap_interval[end], c="k")
    xlim(0., 1.)
    ylim(ylims...)
    # if i > (N-1)÷2+1
    #     ax.set_yticklabels([])
    # end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, (N-1)÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

### Close-up

In [ ]:
start_idx = findfirst(x -> isapprox(x, 0.8, atol=1e-3), coarse_times)
end_idx = findfirst(x -> isapprox(x, 0.85, atol=1e-3), coarse_times)
ts = coarse_times[start_idx:end_idx];

In [ ]:
npts = length(ts)
ωs = [(ω - 1 - npts/2)/(ts[end] - ts[1]) for ω in 1:npts]
fft_f = fftshift(fft(all_flucs[5, start_idx:end_idx]))
spec = abs.(fft_f) ./ maximum(abs.(fft_f));

In [ ]:
figure(figsize=(11, 3))
subplot(131)
plot(coarse_times, all_flucs[3, :], "-")
# xlim(0.75, 0.80)

subplot(132)
# plot(ts, all_flucs[5, start_idx:end_idx], "-")
plot(coarse_times, all_flucs[7, :], "-")

subplot(133)
# plot(ωs, spec)
plot(coarse_times, all_flucs[3, :] .+ all_flucs[7, :] .- mean(all_flucs, dims=1)[1, :], "-")
ylim(0, 0.1)
tight_layout()

## Spectra

In [ ]:
# npts_diag = 16
# τ_final = 1500.
# spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, npts_diag=npts_diag, rtol=1e-6, atol=1e-6);

In [ ]:
T_final = 32768.
T_final = 512.
npts_diag = 16
# T_diags = T_final .* range(0.5, 1.0, npts_diag+1)
T_diags = T_final .* [0.70, 0.72]
# T_diags = T_final .* exact_times[small_idxs[1:findfirst(x -> x == gap_idx, small_idxs) + 1]] 
τ_final = T_final;
# τ_final = 8192.;
# τ_final = 2048.;

In [ ]:
# _, spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, T_diags, rtol=1e-3, atol=1e-4);
_, spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, T_diags, rtol=1e-4, atol=1e-6);

# spec_sum = []
# for k in 1:length(T_diags)
#     push!(spec_sum, spectral_fft(spectral_sum(spectral_sols[k])))
# end

In [ ]:
_, spectral_sums = evolve_spectral_sum(mf_problem, T_final, τ_final, T_diags, rtol=1e-4, atol=1e-6);

spec_sum = []
for k in 1:length(T_diags)
    push!(spec_sum, spectral_fft(spectral_sums[k])) 
    # ωs = h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final))
    # spec = h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final))
    # push!(spec_sum, (ωs, spec))
end

In [ ]:
spectral_data = Dict();

In [ ]:
for spin_idx in 1:N-1
    spectral_data[spin_idx] = spectrum_vs_T(spin_idx, spectral_sols)
end

In [ ]:
spin_idx = top_idx

# map data to functions
spectral_fs = []
for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))
    push!(spectral_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
plot_data = collect(enumerate(zip(spectral_data[spin_idx]...)))
num_plots = length(plot_data)
xlim_vals = (-0.01, 0.2)
# fig, axs = subplots(figsize=(5, 12), nrows=num_plots, ncols=1)
fig, axs = subplots(figsize=(5, 1.5*num_plots), nrows=num_plots, ncols=1)

# T_diags = range(0, 1, npts_diag+1)
for (k, (ωs, spec)) in plot_data
    ax = axs[k] # subplot(num_plots, 1, k)     

    ax.plot(spec_sum[k]..., "-C2")

    ax.plot(ωs, spec, "-", label=@sprintf("\$T=%0.3f\$", T_diags[k] ./ T_final))
    ax.axvline(mingap / (2pi), ls="--", c="k", alpha=0.2)
    ax.axvline(bogo_spec[N, gap_idx] / (2pi), ls="--", c="k", alpha=0.2)
    # max_loc = ωs[findfirst(x->x==maximum(spec), spec)]
    # axvline.([max_loc], c="r", ls="--")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 1)  
    ax.set_xticks(range(xlim_vals..., 5))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    ax.legend(frameon=false, handlelength=0, fontsize=10)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 5), digits=3))

# fig.text(0.0, 0.5, @sprintf("\$\\rho_{%i%i}(T, \\omega)\$", spin_idx, spin_idx), va="center", rotation="vertical")
subplots_adjust(wspace=0.5, hspace=0.16)
# savefig("../plots/" * @sprintf("spectrum_spin_%02i_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", spin_idx, idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
# T_diags = T_final .* range(0.5, 1.0, npts_diag+1)
# @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[4] / T_final)

In [ ]:
omega_range = range(-0.02, 0.3, 1001) |> collect
data_2D = [map(spectral_fs[k], omega_range) for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))];

In [ ]:
cmap = "gist_yarg"

figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, T_diags ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=0.1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
# cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\omega\$")
ax.set_ylabel("\$T\$")
ax.set_xlim(omega_range[1], omega_range[end])
ax.set_ylim(T_diags[1] ./ T_final, T_diags[end] ./ T_final)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

In [ ]:
data_2D_f = interpolate((omega_range, T_diags), reduce(hcat, data_2D), Gridded(Linear()));
interp_data_2D = [[data_2D_f(ω, T) for ω in omega_range] for T in range(T_diags[1], T_diags[end], 1001)];

In [ ]:
figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, range(T_diags[1], T_diags[end], 1001) ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, interp_data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
ax.set_xlim(omega_range[1], omega_range[end])
# ax.set_ylim(0, 1)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

In [ ]:
iidx = 1
plot(spectral_sols[iidx].t, map(t -> spectral_sols[iidx](t)[2, 2], spectral_sols[iidx].t))
xlim(1000, 1200)
# ylim(-100, 100)

## Load Spectrum

In [ ]:
T_final = 32768.
tol = 1e-6
τ_final = 2048.

npts_diag = 16
T_diags = T_final .* range(0.5, 1.0, npts_diag+1);

In [ ]:
# mean-field solution and energy
J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat)

mf_sol = h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
sigma_star = sign.(mf_sol)
h = mf_problem.local_fields
J = mf_problem.couplings
E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

# gap and spectra
λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals")

println(seed, ": " , E_star .- λ[1, end])

gap = λ[2, :] .- λ[1, :]    
exact_times = range(0, 1, 33)
gap_idx = findfirst(x -> x == minimum(gap), gap)
gaploc = exact_times[gap_idx] 
println(gap[gap_idx], ", ", gaploc)
gap_idx = gap_idx - 16 # we use only the interval [0.5, 1.0] with 16+1 points

ωs = []
spec_sum = []
for k in 1:17
# for k in [1]
    push!(ωs, h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final)))
    push!(spec_sum, h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final)))
end   


In [ ]:
figure(figsize=(4, 3))
ax = subplot(111)
axvline(gap[gap_idx + 16], ls="--") 
# for k in 1:17
for k in gap_idx-2:gap_idx
# for k in [gap_idx]
    plot(ωs[k], spec_sum[k])
end
xlim(-0.01, 0.2)
# xlim(-0.01, 0.01)
# ylim(-0., 0.05)
xlabel("\$\\nu\$")
ylabel("Spectrum", fontsize=12)
tight_layout()